In [1]:
import numpy as np
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/TataMotors_historical_data (1).csv")
data

,Date,Open,High,Low,Close,Volume
0,2022-08-01T09:15:00+05:30,451.95,454.00,451.05,453.40,297050
1,2022-08-01T09:16:00+05:30,453.55,454.80,453.40,454.35,171898
2,2022-08-01T09:17:00+05:30,454.30,454.50,453.35,453.35,183982
3,2022-08-01T09:18:00+05:30,453.30,453.70,452.70,453.35,178678
4,2022-08-01T09:19:00+05:30,453.30,453.90,453.05,453.75,106322
...,...,...,...,...,...,...
92245,2023-07-31T15:25:00+05:30,644.80,644.80,644.15,644.15,19116
92246,2023-07-31T15:26:00+05:30,644.35,644.70,644.15,644.70,22111
92247,2023-07-31T15:27:00+05:30,644.70,644.70,644.35,644.40,24553
92248,2023-07-31T15:28:00+05:30,644.40,644.65,642.90,644.00,67930


In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load your historical stock price data into the 'df' DataFrame

# Define RSI Trading Strategy
import pandas as pd

# Load your financial data into a DataFrame (assuming you have it)
data = pd.read_csv('/content/drive/MyDrive/TataMotors_historical_data (1).csv')


#Define RSI strategy
#parameters
rsi_window = 14
rsi_buy_threshold = 30
rsi_sell_threshold = 70

# Calculate RSI
data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
data['Change'] = data['Close'].diff()
data['Gain'] = data['Change'].apply(lambda x: x if x > 0 else 0)
data['Loss'] = -data['Change'].apply(lambda x: x if x < 0 else 0)
data['Average_Gain'] = data['Gain'].rolling(window=rsi_window).mean()
data['Average_Loss'] = data['Loss'].rolling(window=rsi_window).mean()
data['RS'] = data['Average_Gain'] / data['Average_Loss']
data['RSI'] = 100 - (100 / (1 + data['RS']))

# Generate Buy and Sell signals based on RSI thresholds
data['RSI_Buy_Signal'] = data['RSI'] < rsi_buy_threshold
data['RSI_Sell_Signal'] = data['RSI'] > rsi_sell_threshold

# Define Moving Average Crossover Strategy
# parameters
short_period = 50
long_period = 200

# Calculate Short and Long Moving Averages
data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
data['Short_MA'] = data['Close'].rolling(window=short_period).mean()
data['Long_MA'] = data['Close'].rolling(window=long_period).mean()

# Generate Buy and Sell signals based on Moving Average Crossovers
data['SMA_Crossover_Buy_Signal'] = data['Short_MA'] > data['Long_MA']
data['SMA_Crossover_Sell_Signal'] = data['Short_MA'] <= data['Long_MA']


# Define Bollinger Bands strategy
#aprmeters
window = 20
upper_band_multiplier = 2
lower_band_multiplier = 2

# Calculate Rolling Standard Deviation and Middle Band
data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
data['Rolling_STD'] = data['Close'].rolling(window=window).std()
data['Middle_Band'] = data['Close'].rolling(window=window).mean()

# Calculate Upper and Lower Bollinger Bands
data['Upper_Band'] = data['Middle_Band'] + upper_band_multiplier * data['Rolling_STD']
data['Lower_Band'] = data['Middle_Band'] - lower_band_multiplier * data['Rolling_STD']

# Generate Buy and Sell signals based on Bollinger Bands
data['Bollinger_Band_Buy_Signal'] = data['Close'] < data['Lower_Band']
data['Bollinger_Band_Sell_Signal'] = data['Close'] > data['Upper_Band']


# Define MACD Trading Strategy
#parameters
short_window = 12
long_window = 26
signal_window = 9

# Calculate the Short-Term and Long-Term Moving Averages
data['Short_MA'] = data['Close'].rolling(window=short_window).mean()
data['Long_MA'] = data['Close'].rolling(window=long_window).mean()

# Calculate the MACD Line and Signal Line
data['MACD_Line'] = data['Short_MA'] - data['Long_MA']
data['Signal_Line'] = data['MACD_Line'].rolling(window=signal_window).mean()

# Generate Buy and Sell signals based on MACD crossovers
data['MACD_Buy_Signal'] = data['MACD_Line'] > data['Signal_Line']
data['MACD_Sell_Signal'] = data['MACD_Line'] < data['Signal_Line']

# Define RVI Indicator Strategy
#parameters
rvi_window = 14
signal_window = 9
overbought_threshold = 70
oversold_threshold = 30

# Calculate the typical price and its difference
data['Typical_Price'] = (data['High'] + data['Low'] + 2 * data['Close']) / 4
data['Price_Diff'] = data['Typical_Price'].diff()

# Calculate the RVMA (Relative Vigor Moving Average)
data['Positive_Price_Diff'] = data['Price_Diff'].where(data['Price_Diff'] > 0, 0)
data['Negative_Price_Diff'] = -data['Price_Diff'].where(data['Price_Diff'] < 0, 0)
data['RVMA_Positive'] = data['Positive_Price_Diff'].rolling(window=rvi_window).mean()
data['RVMA_Negative'] = data['Negative_Price_Diff'].rolling(window=rvi_window).mean()

# Calculate the RVI (Relative Vigor Index)
data['RVI'] = data['RVMA_Positive'] / (data['RVMA_Positive'] + data['RVMA_Negative'])

# Calculate the Signal Line as a simple moving average of RVI
data['Signal_Line'] = data['RVI'].rolling(window=signal_window).mean()

# Generate Buy and Sell signals based on RVI and Signal Line thresholds
data['RVI_Buy_Signal'] = data['RVI'] > oversold_threshold
data['RVI_Sell_Signal'] = data['RVI'] < overbought_threshold

# Define CCI Strategy
#parameters
cci_window = 20
overbought_threshold = 100
oversold_threshold = -100

# Calculate Typical Price and Moving Average of Typical Price
data['Typical_Price'] = (data['High'] + data['Low'] + data['Close']) / 3
data['MA_Typical_Price'] = data['Typical_Price'].rolling(window=cci_window).mean()

# Calculate Mean Deviation
data['Mean_Deviation'] = abs(data['Typical_Price'] - data['MA_Typical_Price']).rolling(window=cci_window).mean()

# Calculate Commodity Channel Index (CCI)
data['CCI'] = (data['Typical_Price'] - data['MA_Typical_Price']) / (0.015 * data['Mean_Deviation'])

# Implement the strategy logic based on CCI thresholds
data['CCI_Buy_Signal'] = data['CCI'] < oversold_threshold
data['CCI_Sell_Signal'] = data['CCI'] > overbought_threshold


# Define ADX indicator
#parameters
adx_window = 14
adx_threshold = 25

# Calculate True Range
data['HL'] = data['High'] - data['Low']
data['HC'] = abs(data['High'] - data['Close'].shift(1))
data['LC'] = abs(data['Low'] - data['Close'].shift(1))
data['TR'] = data[['HL', 'HC', 'LC']].max(axis=1)

# Calculate Directional Movement
data['DM_plus'] = np.where((data['High'] - data['High'].shift(1)) > (data['Low'].shift(1) - data['Low']), data['High'] - data['High'].shift(1), 0)
data['DM_minus'] = np.where((data['Low'].shift(1) - data['Low']) > (data['High'] - data['High'].shift(1)), data['Low'].shift(1) - data['Low'], 0)

# Calculate Smoothed Directional Movement
data['Smooth_DM_plus'] = data['DM_plus'].rolling(window=adx_window).mean()
data['Smooth_DM_minus'] = data['DM_minus'].rolling(window=adx_window).mean()

# Calculate Directional Indexes (DI+ and DI-)
data['DI_plus'] = (data['Smooth_DM_plus'] / data['TR']) * 100
data['DI_minus'] = (data['Smooth_DM_minus'] / data['TR']) * 100

# Calculate Average Directional Index (ADX)
data['DX'] = (abs(data['DI_plus'] - data['DI_minus']) / (data['DI_plus'] + data['DI_minus'])) * 100
data['ADX'] = data['DX'].rolling(window=adx_window).mean()

# Implement the strategy logic based on ADX threshold
data['ADX_Buy_Signal'] = data['ADX'] > adx_threshold
data['ADX_Sell_Signal'] = data['ADX'] <= adx_threshold

# Define OBV indicator
#parameters
obv_values = []
for idx, row in data.iterrows():
    if row['Close'] > 0:
        obv_values.append(row['Volume'])
    else:
        obv_values.append(-row['Volume'])

data['OBV'] = obv_values

# Generate Buy and Sell Signals based on OBV
data['OBV_Buy_Signal'] = data['OBV'].diff() > 0
data['OBV_Sell_Signal'] = data['OBV'].diff() < 0

# Handle NaN values in the first row
data['OBV_Buy_Signal'] = data['OBV_Buy_Signal'].fillna(False)
data['OBV_Sell_Signal'] = data['OBV_Sell_Signal'].fillna(False)

# Define ATR indicator
#parameters
atr_window = 14
atr_threshold = 0.01

# Calculate True Range (TR)
data['HL'] = data['High'] - data['Low']
data['HC'] = abs(data['High'] - data['Close'].shift(1))
data['LC'] = abs(data['Low'] - data['Close'].shift(1))
data['TR'] = data[['HL', 'HC', 'LC']].max(axis=1)

# Calculate Average True Range (ATR)
data['ATR'] = data['TR'].rolling(window=atr_window).mean()

# Generate Buy and Sell Signals based on ATR threshold
data['ATR_Buy_Signal'] = data['ATR'] > atr_threshold
data['ATR_Sell_Signal'] = data['ATR'] <= atr_threshold

# Define Stochastic Oscillator
#parameters
k_period = 14
d_period = 3
overbought_threshold = 80
oversold_threshold = 20

# Calculate %K and %D
data['L14'] = data['Low'].rolling(window=k_period).min()
data['H14'] = data['High'].rolling(window=k_period).max()
data['%K'] = ((data['Close'] - data['L14']) / (data['H14'] - data['L14'])) * 100
data['%D'] = data['%K'].rolling(window=d_period).mean()

# Generate Buy and Sell Signals based on %K and %D thresholds
data['Stochastic_Buy_Signal'] = data['%K'] < oversold_threshold
data['Stochastic_Sell_Signal'] = data['%K'] > overbought_threshold

# Create the target variable (Profitability)
data['BuyPrice'] = data['Close'].shift(1)
data['ProfitPercentage'] = (data['Close'] - data['BuyPrice']) / data['BuyPrice'] * 100
data.fillna(0, inplace=True)

# Split data into training and testing sets (80% for training, 20% for testing)
train_size = int(0.8 * len(data))
train_data = data[:train_size]
test_data = data[train_size:]

features = {
    'RSI': ['RSI_Buy_Signal', 'RSI_Sell_Signal'],
    'Moving Average Crossover': ['SMA_Crossover_Buy_Signal', 'SMA_Crossover_Sell_Signal'],
    'Bollinger Band': ['Bollinger_Band_Buy_Signal', 'Bollinger_Band_Sell_Signal'],
    'MACD strategy': ['MACD_Buy_Signal', 'MACD_Sell_Signal'],
    'RVI Indicator': ['RVI_Buy_Signal', 'RVI_Sell_Signal'],
    'CCI Strategy': ['CCI_Buy_Signal','CCI_Sell_Signal'],
    'ADX Indicator': ['ADX_Buy_Signal','ADX_Sell_Signal'],
    'OBV Indicator': ['OBV_Buy_Signal','OBV_Sell_Signal'],
    'ATR indicator': ['ATR_Buy_Signal','ATR_Sell_Signal'],
    'Stochastic oscillator': ['Stochastic_Buy_Signal','Stochastic_Sell_Signal']
}

performance_metrics = {}

for strategy, strategy_features in features.items():
    X_train = train_data[strategy_features]
    y_train = train_data['ProfitPercentage']
    X_test = test_data[strategy_features]
    y_test = test_data['ProfitPercentage']
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    r2 = r2_score(y_test, y_pred)

    performance_metrics[strategy] = {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2': r2
    }

print("Performance Metrics for each trading strategy:")
for strategy, metrics in performance_metrics.items():
    print(f"{strategy}:")
    print(f"  MAE: {metrics['MAE']:.2f}")
    print(f"  MSE: {metrics['MSE']:.2f}")
    print(f"  RMSE: {metrics['RMSE']:.2f}")
    print(f"  R-squared (R2): {metrics['R2']:.2f}\n")

# Select the best trading strategy based on the combination of performance metrics
best_strategy = min(performance_metrics, key=lambda k: (
    performance_metrics[k]['MAE'],
    performance_metrics[k]['MSE'],
    performance_metrics[k]['RMSE'],
    performance_metrics[k]['R2']
))

print(f"Best Trading Strategy based on combined performance metrics: {best_strategy}")


Performance Metrics for each trading strategy:
RSI:
  MAE: 0.05
  MSE: 0.00
  RMSE: 0.07
  R-squared (R2): 0.05

Moving Average Crossover:
  MAE: 0.05
  MSE: 0.01
  RMSE: 0.07
  R-squared (R2): -0.00

Bollinger Band:
  MAE: 0.04
  MSE: 0.00
  RMSE: 0.07
  R-squared (R2): 0.15

MACD strategy:
  MAE: 0.05
  MSE: 0.01
  RMSE: 0.07
  R-squared (R2): 0.00

RVI Indicator:
  MAE: 0.05
  MSE: 0.01
  RMSE: 0.07
  R-squared (R2): -0.00

CCI Strategy:
  MAE: 0.04
  MSE: 0.00
  RMSE: 0.07
  R-squared (R2): 0.06

ADX Indicator:
  MAE: 0.05
  MSE: 0.01
  RMSE: 0.07
  R-squared (R2): -0.00

OBV Indicator:
  MAE: 0.05
  MSE: 0.01
  RMSE: 0.07
  R-squared (R2): -0.00

ATR indicator:
  MAE: 0.05
  MSE: 0.01
  RMSE: 0.07
  R-squared (R2): -0.00

Stochastic oscillator:
  MAE: 0.04
  MSE: 0.00
  RMSE: 0.07
  R-squared (R2): 0.15

Best Trading Strategy based on combined performance metrics: Bollinger Band
